In [1]:
import pandas as pd
import numpy as np


import json
from pandas import json_normalize

import datetime as dt

import geopandas as gpd
from geopandas import GeoDataFrame

from shapely.geometry import Point

from openaq import OpenAQ

from pyathena import connect

In [2]:
bbox = [-118.944703049253,
        33.7046743723892,
        -117.646385981235,
        34.823304156602]

In [3]:
client = OpenAQ(api_key='a54a2a8bdc813b4c938b45b3c9bea90f7054cfbdfaa26e8c7709b510bf9b3c7c')
response = client.locations.list(
    bbox=bbox,
    limit=1000
)
data = response.dict()
stations_gdf = json_normalize(data['results'])

In [4]:
sensors_gdf = stations_gdf[['id','sensors','datetime_last.utc']]

In [5]:
sensors_gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 432 non-null    int64 
 1   sensors            432 non-null    object
 2   datetime_last.utc  428 non-null    object
dtypes: int64(1), object(2)
memory usage: 10.3+ KB


There are several missing datetimes - I'll drop them

In [6]:
sensors_gdf.dropna(inplace=True)

/tmp/ipykernel_240059/594373066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensors_gdf.dropna(inplace=True)


In [7]:
sensors_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 428 entries, 0 to 431
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 428 non-null    int64 
 1   sensors            428 non-null    object
 2   datetime_last.utc  428 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.4+ KB


In [8]:
sensors_gdf['datetime_last.utc'] = pd.to_datetime(sensors_gdf['datetime_last.utc'])

/tmp/ipykernel_240059/216471590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensors_gdf['datetime_last.utc'] = pd.to_datetime(sensors_gdf['datetime_last.utc'])


In [9]:
sensors_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 428 entries, 0 to 431
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 428 non-null    int64              
 1   sensors            428 non-null    object             
 2   datetime_last.utc  428 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(1)
memory usage: 13.4+ KB


In [10]:
sensors_gdf = sensors_gdf.explode('sensors')

In [11]:
sensors_gdf['pollutant'] = [sensors_gdf.iloc[i]['sensors']['parameter']['display_name'] for i in range(len(sensors_gdf))]

In [12]:
sensors_gdf['sensor_id'] = [sensors_gdf.iloc[i]['sensors']['id'] for i in range(len(sensors_gdf))]

In [13]:
sensors_gdf.drop(columns=['sensors'],inplace=True)

In [14]:
sensors_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1955 entries, 0 to 431
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 1955 non-null   int64              
 1   datetime_last.utc  1955 non-null   datetime64[ns, UTC]
 2   pollutant          1955 non-null   object             
 3   sensor_id          1955 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 76.4+ KB


In [15]:
sensors_gdf = sensors_gdf.sort_values(by='datetime_last.utc')

In [16]:
sensors_gdf.head()

,id,datetime_last.utc,pollutant,sensor_id
11,1564,2016-11-09 21:00:00+00:00,O₃,2754
7,1268,2016-11-09 21:00:00+00:00,O₃,2274
6,1247,2016-11-09 21:00:00+00:00,O₃,2244
13,1585,2016-11-09 21:00:00+00:00,O₃,2794
13,1585,2016-11-09 21:00:00+00:00,PM10,2793


In [17]:
sensors_gdf[(sensors_gdf['pollutant'] == 'PM2.5') & (sensors_gdf['datetime_last.utc'] > '2025-01-01')]

,id,datetime_last.utc,pollutant,sensor_id
278,1008670,2025-01-08 21:05:15+00:00,PM2.5,2001226
332,3363677,2025-01-11 00:00:00+00:00,PM2.5,11986985
107,947182,2025-01-27 15:26:14+00:00,PM2.5,2000977
328,3279444,2025-04-26 03:00:00+00:00,PM2.5,11380180
357,3402588,2025-05-09 19:00:00+00:00,PM2.5,12331748
...,...,...,...,...
106,947180,2025-10-27 21:07:02+00:00,PM2.5,2000718
112,947187,2025-10-27 21:07:04+00:00,PM2.5,2000723
289,1533718,2025-10-27 21:07:04+00:00,PM2.5,6895834
95,947168,2025-10-27 21:07:05+00:00,PM2.5,2000499


https://docs.python.org/3/library/datetime.html#datetime.datetime.now

In [18]:
today = dt.datetime.now(dt.timezone.utc)

In [19]:
one_week_ago = today - dt.timedelta(weeks=1)

In [20]:
active_pm2_5 = sensors_gdf[(sensors_gdf['pollutant'] == 'PM2.5') & (sensors_gdf['datetime_last.utc'] > one_week_ago)]

In [21]:
active_pm2_5 = active_pm2_5.sort_values(by='datetime_last.utc')

In [22]:
active_pm2_5.head()

,id,datetime_last.utc,pollutant,sensor_id
375,3778909,2025-10-24 00:00:00+00:00,PM2.5,14152560
397,5217366,2025-10-27 17:00:00+00:00,PM2.5,13704200
19,5791,2025-10-27 20:00:00+00:00,PM2.5,15731
313,2812720,2025-10-27 20:22:23+00:00,PM2.5,8967463
427,5831928,2025-10-27 20:50:10+00:00,PM2.5,14149760


In [23]:
sensors_gdf[sensors_gdf['id'] == 3778909]

,id,datetime_last.utc,pollutant,sensor_id
375,3778909,2025-10-24 00:00:00+00:00,PM1,14152559
375,3778909,2025-10-24 00:00:00+00:00,PM2.5,14152560
375,3778909,2025-10-24 00:00:00+00:00,Temperature (C),14152562
375,3778909,2025-10-24 00:00:00+00:00,PM0.3 count,14152563
375,3778909,2025-10-24 00:00:00+00:00,RH,14152561


In [24]:
active_pm2_5[active_pm2_5['sensor_id'] == 14152560]

,id,datetime_last.utc,pollutant,sensor_id
375,3778909,2025-10-24 00:00:00+00:00,PM2.5,14152560


In [25]:
active_pm2_5[active_pm2_5['id'] == 3778909]

,id,datetime_last.utc,pollutant,sensor_id
375,3778909,2025-10-24 00:00:00+00:00,PM2.5,14152560


In [26]:
active_pm2_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 375 to 173
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 310 non-null    int64              
 1   datetime_last.utc  310 non-null    datetime64[ns, UTC]
 2   pollutant          310 non-null    object             
 3   sensor_id          310 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 12.1+ KB


The `id` column can now be passed in to an Athena query

In [27]:
station_IDs = list(active_pm2_5['id'])

In [28]:
station_IDs[:10]

[3778909,
 5217366,
 5791,
 2812720,
 5831928,
 2812577,
 5831929,
 5831930,
 2812652,
 947273]

In [29]:
sensor_IDs = list(active_pm2_5['sensor_id'])
print(sensor_IDs[:5])

[14152560, 13704200, 15731, 8967463, 14149760]


## Athena

In [30]:
cursor = connect(
    s3_staging_dir="s3://la-openaq-athena-results-us-east-1/temp/",
    region_name="us-east-1"
).cursor()

cursor.execute("DROP TABLE openaq_db.openaqMeasurements;")

In [32]:
create_table = """
CREATE EXTERNAL TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
PARTITIONED BY (locationid STRING, year STRING, month STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('serialization.format'=',','field.delim'=',')
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor.execute(create_table)

month is handled as `str(today.month).zfill(2)` to ensure March -> 04

In [33]:
for location in station_IDs:
    cursor.execute(f"""
    ALTER TABLE openaq_db.openaqMeasurements ADD
    PARTITION (year='{today.year}', month='{str(today.month).zfill(2)}', locationid='{location}')
    LOCATION 's3://openaq-data-archive/records/csv.gz/locationid={location}/year={today.year}/month={str(today.month).zfill(2)}/';
    """)

OperationalError: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. AlreadyExistsException(message:Partition already exists.)

In [34]:
cursor.execute(f"""
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY sensors_id
               ORDER BY datetime DESC
           ) AS rn
    FROM openaq_db.openaqMeasurements
    WHERE year = '{today.year}'
      AND month = '{str(today.month).zfill(2)}'
      AND sensors_id IN {tuple(sensor_IDs)}
) sub
WHERE rn = 1
ORDER BY datetime DESC;
"""
)
records = cursor.fetchall()

In [35]:
records[3]

(8236,
 24000,
 'North Holywood-8236',
 '2025-10-24T00:00:00-07:00',
 34.181977,
 -118.363036,
 'pm25',
 'µg/m³',
 6.5,
 '8236',
 '2025',
 '10',
 1)

In [ ]:
la_df = pd.DataFrame(data = records, columns=[
    "location_id", "sensors_id", "location", "datetime",
    "lat", "lon", "parameter", "units", "value",
    "partition_locationid", "year", "month", "day"
])
la_gdf = gpd.GeoDataFrame(
    la_df,
    geometry=[Point(xy) for xy in zip(la_df.lon, la_df.lat)],
    crs="EPSG:4326"
)


In [ ]:
la_gdf.head()

In [ ]:
la_gdf['datetime_utc'] = pd.to_datetime(la_gdf['datetime'])

In [ ]:
la_gdf.drop(columns=['partition_locationid','year','month','day','datetime','lat','lon'],inplace=True)

In [ ]:
la_gdf.info()

In [ ]:
la_gdf.columns[:-2]

In [ ]:
la_gdf.head()

In [ ]:
gdf.to_file('../Data/Outputs/la_gdf.gpkg', driver='GPKG', layer='outliers_la_gdf')

In [2]:
la_gdf = gpd.read_file("../Data/Outputs/la_gdf.gpkg", layer='la_gdf')

In [4]:
la_gdf['value'].mean()

np.float64(20.10575822024202)

Physically, it is impossible for us to have a valid reading with `>1000 µg/m³`. Regardless, we can use the inter-quartile range of the data, and clip anything beyond 1.5x of this middle 50%.

In [12]:
la_gdf['value'].sort_values()

50        0.075208
24        0.601625
26        0.733833
46        1.000000
10        1.969486
          ...     
169      21.780000
52       21.934750
1        21.987625
29       25.218958
258    2810.550000
Name: value, Length: 308, dtype: float64

In [13]:
Q1 = la_gdf['value'].quantile(0.25)
Q3 = la_gdf['value'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

la_gdf_filtered = la_gdf[
    (la_gdf['value'] >= lower_bound) &
    (la_gdf['value'] <= upper_bound)
]


In [14]:
la_gdf_filtered['value'].sort_values()

26      0.733833
46      1.000000
10      1.969486
306     1.986542
163     2.280000
         ...    
55     17.713514
70     18.164306
231    19.170000
19     19.452708
276    20.860000
Name: value, Length: 299, dtype: float64

In [18]:
la_gdf_filtered.to_file('../Data/Outputs/la_gdf.gpkg', driver='GPKG', layer='la_gdf_filtered')